In [2]:
import pandas as pd
import awswrangler as wr
from datetime import datetime

In [3]:
df_uso_cartoes = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/uso_cartoes')
df_uso_cartoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35721396 entries, 0 to 35721395
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   clientid         Int64  
 1   convertedvalue   float64
 2   transactiondate  string 
dtypes: Int64(1), float64(1), string(1)
memory usage: 851.7 MB


In [4]:
df_uso_cartoes.head(3)

,clientid,convertedvalue,transactiondate
0,2395260,63.61,2023-06-05
1,12717494,35.16,2023-06-05
2,5271752,49.32,2023-06-05


In [5]:
df_uso_cartoes_agg = df_uso_cartoes.groupby('clientid').agg({
    'convertedvalue':'mean',
    'transactiondate':'count'
}).rename(columns = {
    'convertedvalue':'ticket_mean_value',
    'transactiondate':'total_transactions'
}).reset_index()

In [6]:
df_uso_cartoes_time  = df_uso_cartoes.groupby('clientid')['transactiondate'].agg(['min', 'max']).reset_index()
df_uso_cartoes_time.columns = ['clientid', 'data_min', 'data_max']

In [7]:
df_uso_cartoes_time['time_diff'] = (pd.to_datetime(df_uso_cartoes_time['data_max']) - pd.to_datetime(df_uso_cartoes_time['data_min'])).dt.days

In [8]:
df_uso_cartoes_time

,clientid,data_min,data_max,time_diff
0,57,2023-06-01,2023-08-01,61
1,59,2023-06-02,2023-07-24,52
2,96,2023-06-02,2023-08-01,60
3,98,2023-06-09,2023-07-31,52
4,106,2023-06-20,2023-07-19,29
...,...,...,...,...
2371178,24739526,2023-08-01,2023-08-01,0
2371179,24739582,2023-08-01,2023-08-01,0
2371180,24739666,2023-08-01,2023-08-01,0
2371181,24739683,2023-08-01,2023-08-01,0


In [9]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_uso_cartoes_time, on = 'clientid')

In [10]:
df_uso_cartoes_agg['mean_time_between_transactions'] = df_uso_cartoes_agg['time_diff'] / df_uso_cartoes_agg['total_transactions']

In [11]:
df_uso_cartoes_agg

,clientid,ticket_mean_value,total_transactions,data_min,data_max,time_diff,mean_time_between_transactions
0,57,735.123925,186,2023-06-01,2023-08-01,61,0.327957
1,59,42.249231,13,2023-06-02,2023-07-24,52,4.000000
2,96,93.015526,38,2023-06-02,2023-08-01,60,1.578947
3,98,22.743571,14,2023-06-09,2023-07-31,52,3.714286
4,106,18.476667,6,2023-06-20,2023-07-19,29,4.833333
...,...,...,...,...,...,...,...
2371178,24739526,400.000000,1,2023-08-01,2023-08-01,0,0.000000
2371179,24739582,101.666667,3,2023-08-01,2023-08-01,0,0.000000
2371180,24739666,4.490000,1,2023-08-01,2023-08-01,0,0.000000
2371181,24739683,18.490000,1,2023-08-01,2023-08-01,0,0.000000


In [12]:
# Persistindo no meio do caminho

wr.s3.to_parquet(df_uso_cartoes_agg,'s3://neon-datascience-sandbox/churn-cartoes/agg')

{'paths': ['s3://neon-datascience-sandbox/churn-cartoes/agg'],
 'partitions_values': {}}

In [13]:
# Lendo dataframe de uso dos cartões

df_uso_cartoes_agg = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/agg')
df_uso_cartoes_agg

,clientid,ticket_mean_value,total_transactions,data_min,data_max,time_diff,mean_time_between_transactions
0,57,735.123925,186,2023-06-01,2023-08-01,61,0.327957
1,59,42.249231,13,2023-06-02,2023-07-24,52,4.000000
2,96,93.015526,38,2023-06-02,2023-08-01,60,1.578947
3,98,22.743571,14,2023-06-09,2023-07-31,52,3.714286
4,106,18.476667,6,2023-06-20,2023-07-19,29,4.833333
...,...,...,...,...,...,...,...
2371178,24739526,400.000000,1,2023-08-01,2023-08-01,0,0.000000
2371179,24739582,101.666667,3,2023-08-01,2023-08-01,0,0.000000
2371180,24739666,4.490000,1,2023-08-01,2023-08-01,0,0.000000
2371181,24739683,18.490000,1,2023-08-01,2023-08-01,0,0.000000


In [14]:
df_dimension = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/dimension')
df_dimension.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18935814 entries, 0 to 18935813
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   clientid        Int64  
 1   presumedincome  float64
 2   age             float64
 3   ageneon         Int64  
dtypes: Int64(2), float64(2)
memory usage: 614.0 MB


In [15]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_dimension, on = 'clientid')
df_uso_cartoes_agg

,clientid,ticket_mean_value,total_transactions,data_min,data_max,time_diff,mean_time_between_transactions,presumedincome,age,ageneon
0,57,735.123925,186,2023-06-01,2023-08-01,61,0.327957,39000.00,31.0,87
1,59,42.249231,13,2023-06-02,2023-07-24,52,4.000000,15000.00,33.0,87
2,96,93.015526,38,2023-06-02,2023-08-01,60,1.578947,3200.00,68.0,86
3,98,22.743571,14,2023-06-09,2023-07-31,52,3.714286,1083.70,42.0,86
4,106,18.476667,6,2023-06-20,2023-07-19,29,4.833333,4253.75,43.0,85
...,...,...,...,...,...,...,...,...,...,...
2191656,24739526,400.000000,1,2023-08-01,2023-08-01,0,0.000000,2222.00,43.0,1
2191657,24739582,101.666667,3,2023-08-01,2023-08-01,0,0.000000,862.00,27.0,1
2191658,24739666,4.490000,1,2023-08-01,2023-08-01,0,0.000000,906.00,30.0,1
2191659,24739683,18.490000,1,2023-08-01,2023-08-01,0,0.000000,4708.00,39.0,1


In [16]:
df_gh = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/gh')
df_gh['clientid'] = df_gh['clientid'].astype("Int64")
df_gh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21867610 entries, 0 to 21867609
Data columns (total 2 columns):
 #   Column    Dtype  
---  ------    -----  
 0   clientid  Int64  
 1   GH_VL     float64
dtypes: Int64(1), float64(1)
memory usage: 354.5 MB


In [17]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_gh, on = 'clientid')
df_uso_cartoes_agg

,clientid,ticket_mean_value,total_transactions,data_min,data_max,time_diff,mean_time_between_transactions,presumedincome,age,ageneon,GH_VL
0,59,42.249231,13,2023-06-02,2023-07-24,52,4.000000,15000.00,33.0,87,4.0
1,96,93.015526,38,2023-06-02,2023-08-01,60,1.578947,3200.00,68.0,86,9.0
2,98,22.743571,14,2023-06-09,2023-07-31,52,3.714286,1083.70,42.0,86,8.0
3,106,18.476667,6,2023-06-20,2023-07-19,29,4.833333,4253.75,43.0,85,6.0
4,115,151.885510,98,2023-06-02,2023-08-01,60,0.612245,3258.57,32.0,85,5.0
...,...,...,...,...,...,...,...,...,...,...,...
2188869,24739526,400.000000,1,2023-08-01,2023-08-01,0,0.000000,2222.00,43.0,1,9.0
2188870,24739582,101.666667,3,2023-08-01,2023-08-01,0,0.000000,862.00,27.0,1,6.0
2188871,24739666,4.490000,1,2023-08-01,2023-08-01,0,0.000000,906.00,30.0,1,1.0
2188872,24739683,18.490000,1,2023-08-01,2023-08-01,0,0.000000,4708.00,39.0,1,9.0


In [18]:
df_churn = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/churn')
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375260 entries, 0 to 375259
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   clientid             375260 non-null  Int64
 1   tempo_sem_transacao  375260 non-null  Int64
 2   churn_flag           375260 non-null  Int64
dtypes: Int64(3)
memory usage: 9.7 MB


In [19]:
df_churn

,clientid,tempo_sem_transacao,churn_flag
0,47814,35,1
1,180086,57,1
2,254615,31,1
3,192177,44,1
4,86246,50,1
...,...,...,...
375255,24141256,52,1
375256,24471188,52,1
375257,24472822,52,1
375258,24462890,52,1


In [20]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_churn, on = 'clientid', how = 'left')

In [21]:
df_uso_cartoes_agg

,clientid,ticket_mean_value,total_transactions,data_min,data_max,time_diff,mean_time_between_transactions,presumedincome,age,ageneon,GH_VL,tempo_sem_transacao,churn_flag
0,59,42.249231,13,2023-06-02,2023-07-24,52,4.000000,15000.00,33.0,87,4.0,<NA>,<NA>
1,96,93.015526,38,2023-06-02,2023-08-01,60,1.578947,3200.00,68.0,86,9.0,<NA>,<NA>
2,98,22.743571,14,2023-06-09,2023-07-31,52,3.714286,1083.70,42.0,86,8.0,<NA>,<NA>
3,106,18.476667,6,2023-06-20,2023-07-19,29,4.833333,4253.75,43.0,85,6.0,<NA>,<NA>
4,115,151.885510,98,2023-06-02,2023-08-01,60,0.612245,3258.57,32.0,85,5.0,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188869,24739526,400.000000,1,2023-08-01,2023-08-01,0,0.000000,2222.00,43.0,1,9.0,<NA>,<NA>
2188870,24739582,101.666667,3,2023-08-01,2023-08-01,0,0.000000,862.00,27.0,1,6.0,<NA>,<NA>
2188871,24739666,4.490000,1,2023-08-01,2023-08-01,0,0.000000,906.00,30.0,1,1.0,<NA>,<NA>
2188872,24739683,18.490000,1,2023-08-01,2023-08-01,0,0.000000,4708.00,39.0,1,9.0,<NA>,<NA>


In [22]:
df_uso_cartoes_agg.churn_flag.fillna(0, inplace = True)

In [23]:
df_uso_cartoes_agg.churn_flag.value_counts()

0    2158630
1      30244
Name: churn_flag, dtype: Int64

In [26]:
df_credito = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/credito')
df_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4264145 entries, 0 to 4264144
Data columns (total 5 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   clientid                 Int64  
 1   limitecreditototal       float64
 2   debitoautomatico         boolean
 3   limitecreditodisponivel  float64
 4   diasatraso               float64
dtypes: Int64(1), boolean(1), float64(3)
memory usage: 142.3 MB


In [28]:
df_credito.head()

,clientid,limitecreditototal,debitoautomatico,limitecreditodisponivel,diasatraso
0,2305010,3060.0,False,2722.33,0.0
1,15957673,2117.0,False,1755.59,0.0
2,20682039,1400.0,True,1400.00,0.0
3,23674087,10.0,True,10.00,0.0
4,15921291,1050.0,True,818.32,0.0


In [30]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_credito, on = 'clientid')
df_uso_cartoes_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187495 entries, 0 to 2187494
Data columns (total 17 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   clientid                        Int64  
 1   ticket_mean_value               float64
 2   total_transactions              Int64  
 3   data_min                        string 
 4   data_max                        string 
 5   time_diff                       Int64  
 6   mean_time_between_transactions  float64
 7   presumedincome                  float64
 8   age                             float64
 9   ageneon                         Int64  
 10  GH_VL                           float64
 11  tempo_sem_transacao             Int64  
 12  churn_flag                      Int64  
 13  limitecreditototal              float64
 14  debitoautomatico                boolean
 15  limitecreditodisponivel         float64
 16  diasatraso                      float64
dtypes: Int64(6), boolean(1), fl

In [27]:
df_account_balance = wr.s3.read_parquet('s3://neon-datascience-sandbox/churn-cartoes/account_balance')
df_account_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10578071 entries, 0 to 10578070
Data columns (total 2 columns):
 #   Column              Dtype  
---  ------              -----  
 0   clientid            Int64  
 1   ACCOUNT_BALANCE_VL  float64
dtypes: Int64(1), float64(1)
memory usage: 171.5 MB


In [29]:
df_account_balance.head()

,clientid,ACCOUNT_BALANCE_VL
0,11347140,610.00
1,11513399,0.00
2,11358216,120020.76
3,11519126,0.00
4,11359904,60.00


In [31]:
df_uso_cartoes_agg = df_uso_cartoes_agg.merge(df_account_balance, on = 'clientid')
df_uso_cartoes_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066132 entries, 0 to 2066131
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   clientid                        Int64  
 1   ticket_mean_value               float64
 2   total_transactions              Int64  
 3   data_min                        string 
 4   data_max                        string 
 5   time_diff                       Int64  
 6   mean_time_between_transactions  float64
 7   presumedincome                  float64
 8   age                             float64
 9   ageneon                         Int64  
 10  GH_VL                           float64
 11  tempo_sem_transacao             Int64  
 12  churn_flag                      Int64  
 13  limitecreditototal              float64
 14  debitoautomatico                boolean
 15  limitecreditodisponivel         float64
 16  diasatraso                      float64
 17  ACCOUNT_BALANCE_VL         

In [32]:
wr.s3.to_parquet(df_uso_cartoes_agg, 's3://neon-datascience-sandbox/churn-cartoes/main')

{'paths': ['s3://neon-datascience-sandbox/churn-cartoes/main'],
 'partitions_values': {}}